plot gradients of latents or activations of convolutional channels

In [1]:
import sys
sys.path.append('/home/xinyiz/c2p_dag/')

import os
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import gaussian, threshold_otsu
from skimage.transform import resize
from skimage import io
import pickle
from skimage.measure import regionprops

import matplotlib.pyplot as plt
import torch
import time
import model.model_cnnvae_conditional
import model.optimizer as optimizer
from sklearn.decomposition import PCA
import pandas as pd
import torchvision

from PIL import Image 

import scanpy
import anndata as ad

In [2]:
nProt=6
holdOutSamples_samples=[['HV1','P22','P14','P27','HV3','P46','P24','P37'],
                        ['HV5','P68','P44','P42','HV7','P15','P55','P59'],
                        ['HV8','P47','P63','P70','HV2','P57','P72','P83'],
                        ['HV4','P16','P18','P62','HV6','P52','P41','P38'],
                        ['HV1','P84','P50','P48','HV2','P22','P56','P27'],
                        ['HV3','P46','P14','P37','HV4','P68','P44','P42'],
                        ['HV5','P15','P24','P59','HV6','P47','P55','P70'],
                        ['HV7','P57','P63','P83','HV8','P16','P18','P62'],
                        ['HV1','P52','P72','P38','HV5','P84','P41','P48'],
                        ['HV2','P22','P50','P27','HV3','P68','P24','P59'],
                        ['HV4','P47','P56','P37','HV7','P16','P44','P70'],
                        ['HV6','P46','P14','P42','HV8','P15','P55','P83']]

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "6"

deShared_centered_2bins=[None]*len(holdOutSamples_samples)
deD_centered_2bins=[None]*len(holdOutSamples_samples)

deShared_centered_2bins_heldout=[None]*len(holdOutSamples_samples)
deD_centered_2bins_heldout=[None]*len(holdOutSamples_samples)


In [3]:
sharedSizes=[1024]
dSpecific_filter=[(200,16)]
pID_type='randInit'
pIDemb_size=64

#VAE settings
seed=3
epochs=5001
saveFreq=100
lr=0.001 #initial learning rate
weight_decay=0 #Weight for L2 loss on embedding matrix.

# batchsize=4
batchsize=256
kernel_size=4
stride=2
padding=1

# fc_dim1=6000
hidden1=64 #Number of channels in hidden layer 1
hidden2=128 
hidden3=256
hidden4=256
hidden5=96
hidden5_xy=4
fc_dim1=96*hidden5_xy*hidden5_xy
fc_dim2=6000

dropout=0.01
kl_weight=0.0000001


name_lord='splitChannels_conditional_lord_withNoise_bce'
modelname_lord='cnn_vae_pbmc_lord'
logsavepath_lord=os.path.join('/data/xinyi/c2p/log/',modelname_lord,name_lord)
modelsavepath_lord=os.path.join('/data/xinyi/c2p/models/',modelname_lord,name_lord)
plotsavepath_lord=os.path.join('/data/xinyi/c2p/plots/',modelname_lord,name_lord)

loadEpoch='3399'
loadEpoch_decoder=372

logsavepath_p_dna=os.path.join(logsavepath_lord,'dna')
modelsavepath_p_dna=os.path.join(modelsavepath_lord,'dna')
plotsavepath_p_dna=os.path.join(plotsavepath_lord,'dna')

logsavepath_p_protein=os.path.join(logsavepath_lord,'protein')
modelsavepath_p_protein=os.path.join(modelsavepath_lord,'protein')
plotsavepath_p_protein=os.path.join(plotsavepath_lord,'protein')

currLatentSize=sharedSizes[0]
dSpecificSize,dfilterSize=dSpecific_filter[0]

dna_cShared=hidden5-dfilterSize
p_cShared=dna_cShared

if modelname_lord=='cnn_vae_pbmc_lord':
    modelcnn_dna_dec = model.model_cnnvae_conditional.CNN_VAE_decode_pIDemb(kernel_size, stride, padding, 1, hidden1, hidden2, hidden3, hidden4, hidden5, fc_dim1,currLatentSize+dSpecificSize,pIDemb_size)
    modelcnn_protein_dec = model.model_cnnvae_conditional.CNN_VAE_decode_pIDemb(kernel_size, stride, padding, 1, hidden1, hidden2, hidden3, hidden4, hidden5, fc_dim1,currLatentSize+dSpecificSize,pIDemb_size)
    modelcnn_dnaShared_dec=model.model_cnnvae_conditional.CNN_VAE_decode_pIDemb(kernel_size, stride, padding, 1, hidden1, hidden2, hidden3, hidden4, hidden5, fc_dim1,currLatentSize,pIDemb_size)
    modelcnn_pShared_dec=model.model_cnnvae_conditional.CNN_VAE_decode_pIDemb(kernel_size, stride, padding, 1, hidden1, hidden2, hidden3, hidden4, hidden5, fc_dim1,currLatentSize,pIDemb_size)
modelcnn_dna_dec.cuda().eval()
modelcnn_protein_dec.cuda().eval()
modelcnn_dnaShared_dec.cuda().eval()
modelcnn_pShared_dec.cuda().eval()

batchsize=328
pIDemb_dec=torch.nn.Embedding(nProt, pIDemb_size).cuda()


with open(os.path.join(modelsavepath_p_protein,'pIDemb_'+str(currLatentSize)+'_'+str(dSpecificSize)+'_'+str(loadEpoch)), 'rb') as output:
    pIDemb_dec.weight=pickle.load(output)

with open(os.path.join(modelsavepath_p_dna,'stateDict_'+str(currLatentSize)+'_'+str(dSpecificSize)), 'rb') as output:
    stateDict_train_dna_dec=pickle.load(output)
with open(os.path.join(modelsavepath_p_protein,'stateDict_'+str(currLatentSize)+'_'+str(dSpecificSize)), 'rb') as output:
    stateDict_train_protein_dec=pickle.load(output)
with open(os.path.join(modelsavepath_p_dna,'stateDictShared_'+str(currLatentSize)+'_'+str(dSpecificSize)), 'rb') as output:
    stateDict_train_dnaShared_dec=pickle.load(output)
with open(os.path.join(modelsavepath_p_protein,'stateDictShared_'+str(currLatentSize)+'_'+str(dSpecificSize)), 'rb') as output:
    stateDict_train_proteinShared_dec=pickle.load(output)

modelcnn_dna_dec.load_state_dict(stateDict_train_dna_dec[int(loadEpoch)])
modelcnn_dnaShared_dec.load_state_dict(stateDict_train_dnaShared_dec[int(loadEpoch)])
modelcnn_protein_dec.load_state_dict(stateDict_train_protein_dec[int(loadEpoch)])
modelcnn_pShared_dec.load_state_dict(stateDict_train_proteinShared_dec[int(loadEpoch)])
pIDemb_dec.weight.requires_grad=False

with open(os.path.join(modelsavepath_p_dna,'stateDict_encode_'+str(currLatentSize)+'_'+str(dSpecificSize)), 'rb') as output:
    stateDict_train_dna=pickle.load(output)
with open(os.path.join(modelsavepath_p_protein,'stateDict_encode_'+str(currLatentSize)+'_'+str(dSpecificSize)), 'rb') as output:
    stateDict_train_protein=pickle.load(output)

if modelname_lord=='cnn_vae_pbmc_lord':
    modelcnn_dna = model.model_cnnvae_conditional.CNN_VAE_split_encode_pIDemb(kernel_size, stride, padding, 1, hidden1, hidden2, hidden3, hidden4, hidden5,p_cShared,dna_cShared*hidden5_xy*hidden5_xy, (hidden5-p_cShared)*hidden5_xy*hidden5_xy,currLatentSize,dSpecificSize,nProt,'randInit',pIDemb_size)
    modelcnn_protein = model.model_cnnvae_conditional.CNN_VAE_split_encode_pIDemb(kernel_size, stride, padding, 1, hidden1, hidden2, hidden3, hidden4, hidden5,p_cShared,p_cShared*hidden5_xy*hidden5_xy, (hidden5-p_cShared)*hidden5_xy*hidden5_xy,currLatentSize,dSpecificSize,nProt,'randInit',pIDemb_size)
modelcnn_dna.cuda()
modelcnn_protein.cuda()
modelcnn_dna.load_state_dict(stateDict_train_dna[loadEpoch_decoder])
modelcnn_protein.load_state_dict(stateDict_train_protein[loadEpoch_decoder])







<All keys matched successfully>

In [6]:

pnames=np.array(['cd16', 'cd3', 'cd4', 'cd8', 'gh2ax', 'lamin'])

In [48]:
sourceDir='/data/xinyi/c2p/data/chromark'
segDir=os.path.join(sourceDir,'nuclear_masks')
imgDir=os.path.join(sourceDir,'raw_data')
conditions=['controls','headneck','meningioma', 'glioma']

outSize=128
savename='pathCentered_'+str(outSize)

for h in range(len(holdOutSamples_samples)):
    print(h)
    holdOutSamples=holdOutSamples_samples[h]

    imgsC_all_val={}
    imgsP_all_val={}
    imgNames_all_val={}
    nmco_all_val={}
    pID_all_val={}
    conditions_all_val={}
    for condition_i in conditions:
#         print(condition_i)
        segDir_i=os.path.join(segDir,condition_i)
        imgDir_i=os.path.join(imgDir,condition_i)
        for stain in os.listdir(segDir_i):
#             print(stain)
            segDir_i_stain=os.path.join(segDir_i,stain)
            imgDir_i_stain=os.path.join(imgDir_i,stain)

            segPID2name={}
            for pID_dir in os.listdir(segDir_i_stain):
                pID=pID_dir.split('_')
                segPID2name[pID[0]]=pID_dir
            imgPID2name={}
            for pID_dir in os.listdir(imgDir_i_stain):
                pID=pID_dir.split('_')
                imgPID2name[pID[0]]=pID_dir
            for pID in segPID2name.keys():
                if condition_i=='meningioma' and stain=='dapi_gh2ax_lamin_cd3' and pID=='P33': #skipping incorrect images
                    continue
                if pID in holdOutSamples:
                    print('hold out: '+pID)
                    continue
#                 print(pID)
                if pID not in imgPID2name:
#                     print('img not found '+pID)
                    continue
                imgDir_i_stain_p=os.path.join(imgDir_i_stain,imgPID2name[pID])
                segDir_i_stain_p=os.path.join(segDir_i_stain,segPID2name[pID])

                with open(os.path.join(imgDir_i_stain_p,savename+'_imgNames'), 'rb') as output:
                    imgNames=pickle.load(output)
                with open(os.path.join(imgDir_i_stain_p,savename+'_img'), 'rb') as output:
                    img=pickle.load(output)
                with open(os.path.join(imgDir_i_stain_p,'nmco_allfeatures_protein'), 'rb') as output:
                    nmco_p=pickle.load(output) #excluding label column

                stain_list=stain.split('_')
                nImgPerStain=int(img.shape[0]/(len(stain_list)-1))
                for sother in range(1,len(stain_list)):
                    
                    if stain_list[sother] not in imgsP_all_val.keys():
                        pID_all_val[stain_list[sother]]=np.repeat(pID,img.shape[0])
                        imgsC_all_val[stain_list[sother]]=img[:,[0]]
                        imgNames_all_val[stain_list[sother]]=imgNames
                        nmco_all_val[stain_list[sother]]=nmco_p[stain_list[sother]]
                        imgsP_all_val[stain_list[sother]]=img[:,[sother]]
                        conditions_all_val[stain_list[sother]]=np.repeat(condition_i,img.shape[0])
                    else:
                        pID_all_val[stain_list[sother]]=np.concatenate((pID_all_val[stain_list[sother]],np.repeat(pID,img.shape[0])))
                        imgsC_all_val[stain_list[sother]]=np.concatenate((imgsC_all_val[stain_list[sother]],img[:,[0]]),axis=0)
                        imgNames_all_val[stain_list[sother]]=np.concatenate((imgNames_all_val[stain_list[sother]],imgNames))
                        nmco_all_val[stain_list[sother]]=np.concatenate((nmco_all_val[stain_list[sother]],nmco_p[stain_list[sother]]),axis=0)
                        imgsP_all_val[stain_list[sother]]=np.concatenate((imgsP_all_val[stain_list[sother]],img[:,[sother]]),axis=0)
                        conditions_all_val[stain_list[sother]]=np.concatenate((conditions_all_val[stain_list[sother]],np.repeat(condition_i,img.shape[0])))
    
    imgsC_val_allProt={}
    imgsP_val_allProt={}
    imgNames_val_allProt={}
    nmco_val_allProt={}
    pID_val_allProt={}
    conditions_val_allProt={}
    for condition_i in conditions:
#         print(condition_i)
        segDir_i=os.path.join(segDir,condition_i)
        imgDir_i=os.path.join(imgDir,condition_i)
        for stain in os.listdir(segDir_i):
#             print(stain)
            segDir_i_stain=os.path.join(segDir_i,stain)
            imgDir_i_stain=os.path.join(imgDir_i,stain)

            segPID2name={}
            for pID_dir in os.listdir(segDir_i_stain):
                pID=pID_dir.split('_')
                segPID2name[pID[0]]=pID_dir
            imgPID2name={}
            for pID_dir in os.listdir(imgDir_i_stain):
                pID=pID_dir.split('_')
                imgPID2name[pID[0]]=pID_dir
            for pID in segPID2name.keys():
                if condition_i=='meningioma' and stain=='dapi_gh2ax_lamin_cd3' and pID=='P33': #skipping incorrect images
                    continue
                if pID not in holdOutSamples:
                    continue
                if pID not in imgPID2name:
#                     print('img not found '+pID)
                    continue
                imgDir_i_stain_p=os.path.join(imgDir_i_stain,imgPID2name[pID])
                segDir_i_stain_p=os.path.join(segDir_i_stain,segPID2name[pID])

                with open(os.path.join(imgDir_i_stain_p,savename+'_imgNames'), 'rb') as output:
                    imgNames=pickle.load(output)
                with open(os.path.join(imgDir_i_stain_p,savename+'_img'), 'rb') as output:
                    img=pickle.load(output)
                with open(os.path.join(imgDir_i_stain_p,'nmco_allfeatures_protein'), 'rb') as output:
                    nmco=pickle.load(output)

                stain_list=stain.split('_')
                for s in range(1,len(stain_list)):
                    if stain_list[s] not in imgsP_val_allProt.keys():
                        pID_val_allProt[stain_list[s]]=np.repeat(pID,img.shape[0])
                        imgsC_val_allProt[stain_list[s]]=img[:,[0]]
                        imgNames_val_allProt[stain_list[s]]=imgNames
                        nmco_val_allProt[stain_list[s]]=nmco[stain_list[s]]
                        imgsP_val_allProt[stain_list[s]]=img[:,[s]]
                        conditions_val_allProt[stain_list[s]]=np.repeat(condition_i,img.shape[0])
                    else:
                        pID_val_allProt[stain_list[s]]=np.concatenate((pID_val_allProt[stain_list[s]],np.repeat(pID,img.shape[0])))
                        imgsC_val_allProt[stain_list[s]]=np.concatenate((imgsC_val_allProt[stain_list[s]],img[:,[0]]),axis=0)
                        imgNames_val_allProt[stain_list[s]]=np.concatenate((imgNames_val_allProt[stain_list[s]],imgNames))
                        nmco_val_allProt[stain_list[s]]=np.concatenate((nmco_val_allProt[stain_list[s]],nmco[stain_list[s]]),axis=0)
                        imgsP_val_allProt[stain_list[s]]=np.concatenate((imgsP_val_allProt[stain_list[s]],img[:,[s]]),axis=0)
                        conditions_val_allProt[stain_list[s]]=np.concatenate((conditions_val_allProt[stain_list[s]],np.repeat(condition_i,img.shape[0])))

    deShared_centered_2bins[h]={}
    deShared_centered_2bins_heldout[h]={}
    deD_centered_2bins[h]={}
    deD_centered_2bins_heldout[h]={}
    for p in pnames:
        print(p)
        plotsavepath_de_h=os.path.join(plotsavepath_lord,p,str(h))
        if not os.path.exists(plotsavepath_de_h):
            os.mkdir(plotsavepath_de_h)
            
        
        nmco_all_finite=np.copy(nmco_all_val[p])
        nmco_all_finite[np.logical_not(np.isfinite(nmco_all_val[p]))]=0
        nmco_all_finite=nmco_all_finite[:,np.sum(np.isfinite(nmco_all_val[p]),axis=0)>0]
        nmco_val_allProt_np_finite=np.copy(nmco_val_allProt[p][:,np.sum(np.isfinite(nmco_all_val[p]),axis=0)>0])
        nmco_val_allProt_np_finite[np.logical_not(np.isfinite(nmco_val_allProt_np_finite))]=0
        
        ###compute PCA###
        #training samples
        plabels=torch.tensor(np.repeat(pnames.tolist().index(p),imgsP_all_val[p].shape[0])).long()
        print(torch.unique(plabels))
        valIdx=np.arange(imgsP_all_val[p].shape[0])
        latent_train_shared_dna=np.zeros((imgsP_all_val[p].shape[0],sharedSizes[0]))
        latent_train_d_dna=np.zeros((imgsP_all_val[p].shape[0],dSpecificSize))
        nvalBatches=int(np.ceil(valIdx.shape[0]/batchsize))
        with torch.no_grad():
            for i in range(nvalBatches):
                valIdx_i=valIdx[i*batchsize:min((i+1)*batchsize,valIdx.shape[0])]
                valtarget_p=torch.tensor(imgsP_all_val[p][valIdx_i]).cuda().float()
                valInput_ID=plabels[valIdx_i].cuda()
                valIdx_i=torch.tensor(valIdx_i)

                reconShared_dna_l,recon_dna_l= modelcnn_protein(valtarget_p,valInput_ID)

                latent_train_shared_dna[valIdx_i]=reconShared_dna_l.cpu().numpy()
                latent_train_d_dna[valIdx_i]=recon_dna_l.cpu().numpy()

        pca_train_shared_dna=PCA()
        pca_train_d_dna=PCA()
        pca_train_shared_dna.fit(latent_train_shared_dna)
        pca_train_d_dna.fit(latent_train_d_dna)

        ###bin training cells###
        nPC=10
        nSteps=11 
        percentiles=(np.arange(nSteps-1)+1)*100/nSteps
        nCellsPerSample=30
        allIdx=np.arange(imgsP_all_val[p].shape[0])

        latent_shared_pca=pca_train_shared_dna.transform(latent_train_shared_dna)[:,:nPC]
        latent_shared_pca_percentile=np.percentile(latent_shared_pca,percentiles,axis=0)
        latent_shared_pca_max=np.max(latent_shared_pca,axis=0)
        latent_shared_pca_min=np.min(latent_shared_pca,axis=0)
        latent_d_pca=pca_train_d_dna.transform(latent_train_d_dna)[:,:nPC]
        latent_d_pca_percentile=np.percentile(latent_d_pca,percentiles,axis=0)
        latent_d_pca_max=np.max(latent_d_pca,axis=0)
        latent_d_pca_min=np.min(latent_d_pca,axis=0)
        dist2origin_shared=np.square(latent_shared_pca)
        dist2origin_d=np.square(latent_d_pca)

        latent_shared_pca_binID=np.zeros((nPC,latent_shared_pca.shape[0]))-1
        centerIdx_shared_all=[None]*nPC
        latent_d_pca_binID=np.zeros((nPC,latent_d_pca.shape[0]))-1
        centerIdx_d_all=[None]*nPC
        for pc_i in range(nPC):

            #idx of cells at the center of other pcs
            dist2origin_shared_i=np.sum(dist2origin_shared[:,:pc_i],axis=1)+np.sum(dist2origin_shared[:,pc_i+1:],axis=1)
            thresh_shared_i=np.percentile(dist2origin_shared_i,15)
            dist2origin_d_i=np.sum(dist2origin_d[:,:pc_i],axis=1)+np.sum(dist2origin_d[:,pc_i+1:],axis=1)
            thresh_d_i=np.percentile(dist2origin_d_i,15)
            centerIdx_shared_all[pc_i]=dist2origin_shared_i<thresh_shared_i
            centerIdx_d_all[pc_i]=dist2origin_d_i<thresh_d_i
        #     print(np.sum(centerIdx_shared))
        #     print(np.sum(centerIdx_d))
        #     print('\n')

            for sidx in range(nSteps):
                if sidx==0:
                    min_shared_s=latent_shared_pca_min[pc_i]
                    min_d_s=latent_d_pca_min[pc_i]
                else:
                    min_shared_s=latent_shared_pca_percentile[sidx-1,pc_i]
                    min_d_s=latent_d_pca_percentile[sidx-1,pc_i]
                if sidx==nSteps-1:
                    max_shared_s=latent_shared_pca_max[pc_i]
                    max_d_s=latent_d_pca_max[pc_i]
                else:
                    max_shared_s=latent_shared_pca_percentile[sidx,pc_i]
                    max_d_s=latent_d_pca_percentile[sidx,pc_i]

                latent_shared_pca_binID[pc_i,np.logical_and(latent_shared_pca[:,pc_i]>min_shared_s,latent_shared_pca[:,pc_i]<=max_shared_s)]=sidx
                latent_d_pca_binID[pc_i,np.logical_and(latent_d_pca[:,pc_i]>min_d_s,latent_d_pca[:,pc_i]<=max_d_s)]=sidx


        with open(os.path.join(sourceDir,'nmco_allfeatures_names_'+p), 'rb') as output:
            nmco_names=pickle.load(output)
        nmco_names_finite=nmco_names[np.sum(np.isfinite(nmco_all_val[p]),axis=0)>0]

        ###DE of training###
        groupID_shared=np.zeros_like(latent_shared_pca_binID).astype(str)
        groupID_shared[latent_shared_pca_binID<2]=-1
        groupID_shared[latent_shared_pca_binID>8]=1
        groupID_shared[latent_shared_pca_binID==5]='c'
        groupID_d=np.zeros_like(latent_d_pca_binID).astype(str)
        groupID_d[latent_d_pca_binID<2]=-1
        groupID_d[latent_d_pca_binID>8]=1
        groupID_d[latent_d_pca_binID==5]='c'

        deRes_shared_centered=[None]*nPC
        deRes_d_centered=[None]*nPC
        nmco_ad=ad.AnnData((nmco_all_finite-np.min(nmco_all_finite,axis=0,keepdims=True))/(np.max(nmco_all_finite,axis=0,keepdims=True)-np.min(nmco_all_finite,axis=0,keepdims=True)))

        for pc_i in range(nPC):
            print(pc_i)
            nmco_ad.obs[str(pc_i)+'percentileGroups_shared']=groupID_shared[pc_i].astype(str)
            nmco_ad.obs[str(pc_i)+'percentileGroups_d']=groupID_d[pc_i].astype(str)
            nmco_ad_subShared=nmco_ad[centerIdx_shared_all[pc_i]].copy()
            nmco_ad_subD=nmco_ad[centerIdx_d_all[pc_i]].copy()

            if 'c' in np.unique(nmco_ad_subShared.obs[str(pc_i)+'percentileGroups_shared']):
                scanpy.tl.rank_genes_groups(nmco_ad_subShared, str(pc_i)+'percentileGroups_shared', method='t-test',groups=np.intersect1d(np.unique(nmco_ad_subShared.obs[str(pc_i)+'percentileGroups_shared']),['-1','1','c']).tolist())
                deRes_shared_centered[pc_i]={}
                deRes_shared_centered[pc_i]['0']=scanpy.get.rank_genes_groups_df(nmco_ad_subShared,'c')

            if 'c' in np.unique(nmco_ad_subD.obs[str(pc_i)+'percentileGroups_d']):
                scanpy.tl.rank_genes_groups(nmco_ad_subD, str(pc_i)+'percentileGroups_d', method='t-test',groups=np.intersect1d(np.unique(nmco_ad_subD.obs[str(pc_i)+'percentileGroups_d']),['-1','1','c']).tolist())
                deRes_d_centered[pc_i]={}
                deRes_d_centered[pc_i]['0']=scanpy.get.rank_genes_groups_df(nmco_ad_subD,'c')

            if '1' in np.unique(nmco_ad_subShared.obs[str(pc_i)+'percentileGroups_shared']) and '-1' in np.unique(nmco_ad_subShared.obs[str(pc_i)+'percentileGroups_shared']):
                scanpy.tl.rank_genes_groups(nmco_ad_subShared, str(pc_i)+'percentileGroups_shared', method='t-test',groups=['-1','1'])
                deRes_shared_centered[pc_i]['1']=scanpy.get.rank_genes_groups_df(nmco_ad_subShared,'1')
                deRes_shared_centered[pc_i]['-1']=scanpy.get.rank_genes_groups_df(nmco_ad_subShared,'-1')

            if '1' in np.unique(nmco_ad_subD.obs[str(pc_i)+'percentileGroups_d']) and '-1' in np.unique(nmco_ad_subD.obs[str(pc_i)+'percentileGroups_d']):
                scanpy.tl.rank_genes_groups(nmco_ad_subD, str(pc_i)+'percentileGroups_d', method='t-test',groups=['-1','1'])
                deRes_d_centered[pc_i]['1']=scanpy.get.rank_genes_groups_df(nmco_ad_subD,'1')
                deRes_d_centered[pc_i]['-1']=scanpy.get.rank_genes_groups_df(nmco_ad_subD,'-1')

        deShared_centered_2bins[h][p]=deRes_shared_centered
        deD_centered_2bins[h][p]=deRes_d_centered


        ######## held-out PCA #####
        plabels=torch.tensor(np.repeat(pnames.tolist().index(p),imgsP_val_allProt[p].shape[0])).long()
        valIdx=np.arange(imgsP_val_allProt[p].shape[0])
        latent_heldout_shared_dna=np.zeros((imgsP_val_allProt[p].shape[0],sharedSizes[0]))
        latent_heldout_d_dna=np.zeros((imgsP_val_allProt[p].shape[0],dSpecificSize))
        nvalBatches=int(np.ceil(valIdx.shape[0]/batchsize))
        with torch.no_grad():
            for i in range(nvalBatches):
                valIdx_i=valIdx[i*batchsize:min((i+1)*batchsize,valIdx.shape[0])]
                valtarget_p=torch.tensor(imgsP_val_allProt[p][valIdx_i]).cuda().float()
                valInput_ID=plabels[valIdx_i].cuda()
                valIdx_i=torch.tensor(valIdx_i)

                reconShared_dna_l,recon_dna_l= modelcnn_protein(valtarget_p,valInput_ID)

                latent_heldout_shared_dna[valIdx_i]=reconShared_dna_l.cpu().numpy()
                latent_heldout_d_dna[valIdx_i]=recon_dna_l.cpu().numpy()

        latent_shared_pca_heldout=pca_train_shared_dna.transform(latent_heldout_shared_dna)
        plt.scatter(latent_shared_pca[:,0],latent_shared_pca[:,1],s=0.1,c='blue')
        plt.scatter(latent_shared_pca_heldout[:,0],latent_shared_pca_heldout[:,1],s=0.1,c='red')
        plt.savefig(os.path.join(plotsavepath_de_h,'pca_shared_heldout.pdf'))
        plt.close()

        latent_d_pca_heldout=pca_train_d_dna.transform(latent_heldout_d_dna)
        plt.scatter(latent_d_pca[:,0],latent_d_pca[:,1],s=0.1,c='blue')
        plt.scatter(latent_d_pca_heldout[:,0],latent_d_pca_heldout[:,1],s=0.1,c='red')
        plt.savefig(os.path.join(plotsavepath_de_h,'pca_d_heldout.pdf'))
        plt.close()

        nPC=10
        nSteps=11 
        percentiles=(np.arange(nSteps-1)+1)*100/nSteps
        nCellsPerSample=30

        latent_shared_pca_heldout=pca_train_shared_dna.transform(latent_heldout_shared_dna)[:,:nPC]
        latent_d_pca_heldout=pca_train_d_dna.transform(latent_heldout_d_dna)[:,:nPC]
        dist2origin_shared=np.square(latent_shared_pca) #use training distance thresh
        dist2origin_d=np.square(latent_d_pca)#use training distance thresh
        dist2origin_shared_heldout=np.square(latent_shared_pca_heldout) 
        dist2origin_d_heldout=np.square(latent_d_pca_heldout)

        latent_shared_pca_heldout_binID=np.zeros((nPC,latent_shared_pca_heldout.shape[0]))-1
        centerIdx_shared_all=[None]*nPC
        latent_d_pca_heldout_binID=np.zeros((nPC,latent_d_pca_heldout.shape[0]))-1
        centerIdx_d_all=[None]*nPC
        for pc_i in range(nPC):

            #idx of cells at the center of other pcs
            dist2origin_shared_i=np.sum(dist2origin_shared[:,:pc_i],axis=1)+np.sum(dist2origin_shared[:,pc_i+1:],axis=1)
            thresh_shared_i=np.percentile(dist2origin_shared_i,15)
            dist2origin_d_i=np.sum(dist2origin_d[:,:pc_i],axis=1)+np.sum(dist2origin_d[:,pc_i+1:],axis=1)
            thresh_d_i=np.percentile(dist2origin_d_i,15)

            dist2origin_shared_i_heldout=np.sum(dist2origin_shared_heldout[:,:pc_i],axis=1)+np.sum(dist2origin_shared_heldout[:,pc_i+1:],axis=1)
            dist2origin_d_i_heldout=np.sum(dist2origin_d_heldout[:,:pc_i],axis=1)+np.sum(dist2origin_d_heldout[:,pc_i+1:],axis=1)
            centerIdx_shared_all[pc_i]=dist2origin_shared_i_heldout<thresh_shared_i
            centerIdx_d_all[pc_i]=dist2origin_d_i_heldout<thresh_d_i

            for sidx in range(nSteps):
                if sidx==0:
                    min_shared_s=latent_shared_pca_min[pc_i]
                    min_d_s=latent_d_pca_min[pc_i]
                else:
                    min_shared_s=latent_shared_pca_percentile[sidx-1,pc_i]
                    min_d_s=latent_d_pca_percentile[sidx-1,pc_i]
                if sidx==nSteps-1:
                    max_shared_s=latent_shared_pca_max[pc_i]
                    max_d_s=latent_d_pca_max[pc_i]
                else:
                    max_shared_s=latent_shared_pca_percentile[sidx,pc_i]
                    max_d_s=latent_d_pca_percentile[sidx,pc_i]



                latent_shared_pca_heldout_binID[pc_i,np.logical_and(latent_shared_pca_heldout[:,pc_i]>min_shared_s,latent_shared_pca_heldout[:,pc_i]<=max_shared_s)]=sidx
                latent_d_pca_heldout_binID[pc_i,np.logical_and(latent_d_pca_heldout[:,pc_i]>min_d_s,latent_d_pca_heldout[:,pc_i]<=max_d_s)]=sidx

        groupID_shared_heldout=np.zeros_like(latent_shared_pca_heldout_binID).astype(str)
        groupID_shared_heldout[latent_shared_pca_heldout_binID<2]=-1
        groupID_shared_heldout[latent_shared_pca_heldout_binID>8]=1
        groupID_shared_heldout[latent_shared_pca_heldout_binID==5]='c'
        # groupID_shared_heldout[np.logical_and(latent_shared_pca_heldout_binID>3,latent_shared_pca_heldout_binID<7)]='c'
        groupID_d_heldout=np.zeros_like(latent_d_pca_heldout_binID).astype(str)
        groupID_d_heldout[latent_d_pca_heldout_binID<2]=-1
        groupID_d_heldout[latent_d_pca_heldout_binID>8]=1
        groupID_d_heldout[latent_d_pca_heldout_binID==5]='c'
        # groupID_d_heldout[np.logical_and(latent_d_pca_heldout_binID>3,latent_d_pca_heldout_binID<7)]='c'

        ####DE heldout###
        deRes_shared_centered_heldout=[None]*nPC
        deRes_d_centered_heldout=[None]*nPC
        nmco_ad_heldout=ad.AnnData((nmco_val_allProt_np_finite-np.min(nmco_val_allProt_np_finite,axis=0,keepdims=True))/(np.max(nmco_val_allProt_np_finite,axis=0,keepdims=True)-np.min(nmco_val_allProt_np_finite,axis=0,keepdims=True)))

        for pc_i in range(nPC):
            print(pc_i)
            nmco_ad_heldout.obs[str(pc_i)+'percentileGroups_shared']=groupID_shared_heldout[pc_i].astype(str)
            nmco_ad_heldout.obs[str(pc_i)+'percentileGroups_d']=groupID_d_heldout[pc_i].astype(str)
            nmco_ad_heldout_subShared=nmco_ad_heldout[centerIdx_shared_all[pc_i]].copy()
            nmco_ad_heldout_subD=nmco_ad_heldout[centerIdx_d_all[pc_i]].copy()

            if 'c' in np.unique(nmco_ad_heldout_subShared.obs[str(pc_i)+'percentileGroups_shared']):
                scanpy.tl.rank_genes_groups(nmco_ad_heldout_subShared, str(pc_i)+'percentileGroups_shared', method='t-test',groups=np.intersect1d(np.unique(nmco_ad_heldout_subShared.obs[str(pc_i)+'percentileGroups_shared']),['-1','1','c']).tolist())
                deRes_shared_centered_heldout[pc_i]={}
                deRes_shared_centered_heldout[pc_i]['0']=scanpy.get.rank_genes_groups_df(nmco_ad_heldout_subShared,'c')

            if 'c' in np.unique(nmco_ad_heldout_subD.obs[str(pc_i)+'percentileGroups_d']):
                scanpy.tl.rank_genes_groups(nmco_ad_heldout_subD, str(pc_i)+'percentileGroups_d', method='t-test',groups=np.intersect1d(np.unique(nmco_ad_heldout_subD.obs[str(pc_i)+'percentileGroups_d']),['-1','1','c']).tolist())
                deRes_d_centered_heldout[pc_i]={}
                deRes_d_centered_heldout[pc_i]['0']=scanpy.get.rank_genes_groups_df(nmco_ad_heldout_subD,'c')

            if '1' in np.unique(nmco_ad_heldout_subShared.obs[str(pc_i)+'percentileGroups_shared']) and '-1' in np.unique(nmco_ad_heldout_subShared.obs[str(pc_i)+'percentileGroups_shared']):
                scanpy.tl.rank_genes_groups(nmco_ad_heldout_subShared, str(pc_i)+'percentileGroups_shared', method='t-test',groups=['-1','1'])
                deRes_shared_centered_heldout[pc_i]['1']=scanpy.get.rank_genes_groups_df(nmco_ad_heldout_subShared,'1')
                deRes_shared_centered_heldout[pc_i]['-1']=scanpy.get.rank_genes_groups_df(nmco_ad_heldout_subShared,'-1')

            if '1' in np.unique(nmco_ad_heldout_subD.obs[str(pc_i)+'percentileGroups_d']) and '-1' in np.unique(nmco_ad_heldout_subD.obs[str(pc_i)+'percentileGroups_d']):
                scanpy.tl.rank_genes_groups(nmco_ad_heldout_subD, str(pc_i)+'percentileGroups_d', method='t-test',groups=['-1','1'])
                deRes_d_centered_heldout[pc_i]['1']=scanpy.get.rank_genes_groups_df(nmco_ad_heldout_subD,'1')
                deRes_d_centered_heldout[pc_i]['-1']=scanpy.get.rank_genes_groups_df(nmco_ad_heldout_subD,'-1')
        deShared_centered_2bins_heldout[h][p]=deRes_shared_centered_heldout
        deD_centered_2bins_heldout[h][p]=deRes_d_centered_heldout
        print(deShared_centered_2bins_heldout[h].keys())
        
    

0
hold out: HV1
hold out: HV3
hold out: HV1
hold out: HV3
hold out: P14
hold out: P24
hold out: P14
hold out: P24
hold out: P27
hold out: P37
hold out: P37
hold out: P27
hold out: P46
hold out: P22
hold out: P46
hold out: P22
cd16
tensor([0])
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
dict_keys(['cd16'])
cd3
tensor([1])


/tmp/ipykernel_1644638/3527635232.py:244: RuntimeWarning: invalid value encountered in divide
  nmco_ad=ad.AnnData((nmco_all_finite-np.min(nmco_all_finite,axis=0,keepdims=True))/(np.max(nmco_all_finite,axis=0,keepdims=True)-np.min(nmco_all_finite,axis=0,keepdims=True)))


0
1
2
3
4
5
6
7
8
9


/tmp/ipykernel_1644638/3527635232.py:369: RuntimeWarning: invalid value encountered in divide
  nmco_ad_heldout=ad.AnnData((nmco_val_allProt_np_finite-np.min(nmco_val_allProt_np_finite,axis=0,keepdims=True))/(np.max(nmco_val_allProt_np_finite,axis=0,keepdims=True)-np.min(nmco_val_allProt_np_finite,axis=0,keepdims=True)))


0
1
2
3
4
5
6
7
8
9
dict_keys(['cd16', 'cd3'])
cd4
tensor([2])


...

0
1
2
3
4
5
6
7
8
9
dict_keys(['cd16', 'cd3', 'cd4', 'cd8'])
gh2ax
tensor([4])
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
dict_keys(['cd16', 'cd3', 'cd4', 'cd8', 'gh2ax'])
lamin
tensor([5])


/tmp/ipykernel_1644638/3527635232.py:244: RuntimeWarning: invalid value encountered in divide
  nmco_ad=ad.AnnData((nmco_all_finite-np.min(nmco_all_finite,axis=0,keepdims=True))/(np.max(nmco_all_finite,axis=0,keepdims=True)-np.min(nmco_all_finite,axis=0,keepdims=True)))


0
1
2
3
4
5
6
7
8
9


/tmp/ipykernel_1644638/3527635232.py:369: RuntimeWarning: invalid value encountered in divide
  nmco_ad_heldout=ad.AnnData((nmco_val_allProt_np_finite-np.min(nmco_val_allProt_np_finite,axis=0,keepdims=True))/(np.max(nmco_val_allProt_np_finite,axis=0,keepdims=True)-np.min(nmco_val_allProt_np_finite,axis=0,keepdims=True)))


0
1
2
3
4
5
6
7
8
9
dict_keys(['cd16', 'cd3', 'cd4', 'cd8', 'gh2ax', 'lamin'])


In [49]:
plotsavepath_de=os.path.join(plotsavepath_p_protein,'de')
if not os.path.exists(plotsavepath_de):
    os.mkdir(plotsavepath_de)
with open(os.path.join(plotsavepath_de,'de_shared_all_centered_2bins'), 'wb') as output:
    pickle.dump(deShared_centered_2bins,output,pickle.HIGHEST_PROTOCOL)
with open(os.path.join(plotsavepath_de,'de_d_all_centered_2bins'), 'wb') as output:
    pickle.dump(deD_centered_2bins,output,pickle.HIGHEST_PROTOCOL)
    
with open(os.path.join(plotsavepath_de,'de_shared_all_centered_2bins_heldout'), 'wb') as output:
    pickle.dump(deShared_centered_2bins_heldout,output,pickle.HIGHEST_PROTOCOL)
with open(os.path.join(plotsavepath_de,'de_d_all_centered_2bins_heldout'), 'wb') as output:
    pickle.dump(deD_centered_2bins_heldout,output,pickle.HIGHEST_PROTOCOL)
